In [ ]:
"""
1. 패션데이터셋 읽어들이기 : 변수는 기존과 동일
2. 데이터 스케일링 : 변수는 기존과 동일
3. 2차원 데이터로 변경
4. 훈련 : 검증 = 8:2로 분리 : 변수는 기존과 동일
5. 모델 생성 : 변수명 model
6. 입력계층, 은닉계층, 출력계층 생성 및 모델에 추가하기
7. 모델 환경설정
8. 모델 훈련시키기
"""

In [1]:
### 텐서플로우 프레임워크(패키지라고도 칭함)
import tensorflow as tf

### 케라스 라이브러리 불러들이기
from tensorflow import keras

from keras.layers import Input, Dense
from keras.models import Sequential                  # 순차 모델 클래스
from keras.layers import Dense, Dropout, Input       # 완전 연결층(Dense), 드롭아웃(Dropout) 레이어
from keras.regularizers import l2                    # L2 정규화 함수
from keras.callbacks import EarlyStopping, ModelCheckpoint           # 조기 종료 콜백 함수

### 시각화 라이브러리 불러들이기
import matplotlib.pyplot as plt

### 넘파이
import numpy as np

### 훈련 및 테스트 데이터로 분류하는 라이브러리
from sklearn.model_selection import train_test_split


### 딥러닝 랜덤 규칙 정의하기
# - 딥러닝에서의 랜덤 규칙은 항상 일정하게 유지되지는 않음
# - 딥러닝 모델 내부에서 훈련을 위한 데이터를 임의로 추출하여 사용(사람이 관여 안함)
tf.keras.utils.set_random_seed(42)

In [2]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [3]:
train_scaled_255 = train_input / 255
test_scaled_255 = test_input / 255

print(train_scaled_255.shape, train_target.shape)
print(test_scaled_255.shape, test_target.shape)

(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [4]:
train_scaled_2d = train_scaled_255.reshape(-1, 28 * 28)
test_scaled_2d = test_scaled_255.reshape(-1, 28 * 28)

print(train_scaled_2d.shape, train_target.shape)
print(test_scaled_2d.shape, test_target.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [5]:
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled_2d, train_target, test_size=0.2, random_state=42)

print(train_scaled.shape, train_target.shape)
print(val_scaled.shape, val_target.shape)

(48000, 784) (48000,)
(12000, 784) (12000,)


In [6]:
model = Sequential([
    Input(shape=(784, )),
    Dense(units=100, activation="sigmoid"),
    Dense(units=50, activation="relu"),
    Dense(units=10, activation="softmax")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [7]:
sgd = keras.optimizers.SGD(
    momentum=0.9,
    nesterov=True, 
    learning_rate = 0.001
)

model.compile(
    optimizer=sgd,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [8]:
history = model.fit(train_scaled, train_target, validation_data = (val_scaled, val_target), epochs=100, batch_size=128)

print("\n--------------------------------------------------\n")

train_score = model.evaluate(train_scaled, train_target)
val_score = model.evaluate(val_scaled, val_target)

Epoch 1/100
375/375 [==============================] - 2s 2ms/step - loss: 2.0138 - accuracy: 0.4121 - val_loss: 1.7047 - val_accuracy: 0.6213
Epoch 2/100
375/375 [==============================] - 1s 2ms/step - loss: 1.4446 - accuracy: 0.6353 - val_loss: 1.2358 - val_accuracy: 0.6662
Epoch 3/100
375/375 [==============================] - 1s 2ms/step - loss: 1.0946 - accuracy: 0.6860 - val_loss: 0.9903 - val_accuracy: 0.7111
Epoch 4/100
375/375 [==============================] - 1s 2ms/step - loss: 0.9112 - accuracy: 0.7177 - val_loss: 0.8563 - val_accuracy: 0.7268
Epoch 5/100
375/375 [==============================] - 1s 3ms/step - loss: 0.8061 - accuracy: 0.7336 - val_loss: 0.7750 - val_accuracy: 0.7350
Epoch 6/100
375/375 [==============================] - 1s 2ms/step - loss: 0.7400 - accuracy: 0.7445 - val_loss: 0.7223 - val_accuracy: 0.7466
Epoch 7/100
375/375 [==============================] - 1s 2ms/step - loss: 0.6957 - accuracy: 0.7533 - val_loss: 0.6849 - val_accuracy: 0.7535

### 모델 저장 방법

In [ ]:
"""
<모델 저장 방법>
1. 일반적인 저장 : 모델 훈련(fit)이 완료된 결과를 기준으로 합니다.
                 : 사람이 가장 좋은 시점을 튜닝에 의해 주로 판단함
                 : 가중치만 저장, 모델 자체 저장
                 
2. 훈련 중 저장 : 모델 훈련(fit) 중에 가장 좋은 시점(일반화)에 모델을 저장 합니다.
                : 사람이 개입하지 않음. 모델 훈련 중 자동으로 찾아냅니다.
                : 이벤트 명령을 정의해 주어야 합니다.
                  (이벤트를 -> 콜백 이라고하며, 이때 사용되는 기능을 "콜백 함수"라고 칭함)
                : 모델 자체를 저장
"""

### 1. 일반적인 저장 방법

In [ ]:
"""
<1. 가중치만 저장하는 방식>
 - 훈련 중 손실을 줄이기 위해 모델이 스스로 찾아낸 가중치(w) 값들을 저장하는 방식
 - 저장된 가중치를 사용하는 방법
  - 모델 생성 > 저장된 가중치 읽어들이기 > 예측 진행
   (fit은 하지 않습니다. 이미 훈련된 가중치이기 때문에...)
   
<2. 모델 자체를 저장하는 방식>
 - 훈련 최종 결과 모델 자체를 저장하는 방식
 - 가중치 및 설정들에 대한 모든 것을 저장함
 - 저장된 모델을 사용하는 방법
  - 모델 읽어들이기 > 예측 진행
    (별도의 모델 생성 및 fit은 하지 않습니다. 이미 모델자체를 사용하기 때문에)
"""

##### <가중치를 저장해서 사용하는 방식>

In [34]:
### 모델의 가중치만 저장하기
# - 저장시 파일의 확장자는 보통 h5를 사용함
model.save_weights("./model/model_weights.h5")

In [ ]:
### 저장된 가중치를 불러들이기
# - 모델 생성 > 가중치 불러들이기

# 모델 생성
model2 = Sequential([
    Input(shape=(784, )),
    Dense(units=100, activation="sigmoid", name="Input_layer"),
    Dense(units=50, activation="relu", name="Hidden_layer"),
    Dense(units=10, activation="softmax", name="Output_layer")
])

adam = keras.optimizers.Adam(
    learning_rate = 0.001
)

model2.compile(
    optimizer=adam,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

### 모델 불러들이기
model2.load_weights("./model/model_weights.h5")

### 이후부터는 예측으로 사용...


##### <모델 자체 저장하는 방식>

In [11]:
### 모델 자체 저장하기
model.save("./model/model_all.h5")

In [12]:
### 저장된 모델 불러들이기
model_all = keras.models.load_model("./model/model_all.h5")
model_all.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [13]:
### 예측(predict)하기
test_pred = model_all.predict(test_scaled_2d)

313/313 [==============================] - 0s 1ms/step


In [14]:
### 0번째 예측결과만 추출해보기
test_pred[0]

array([2.59314277e-07, 2.73447966e-08, 7.24521442e-07, 1.18061095e-07,
       5.06180913e-06, 1.33823380e-01, 5.04131958e-06, 1.36282161e-01,
       5.25726331e-03, 7.24625945e-01], dtype=float32)

In [15]:
### 0번째 정답 출력하기
test_target[0]

9

In [16]:
### 예측값과 정답 비교하기
# 예측값 10개의 확률데이터 중에 가장 높은 값의 위치값을 반환함
np.argmax(test_pred[0]), test_target[0]

(9, 9)

In [17]:
np.argmax(test_pred[1]), test_target[1]

(2, 2)

In [18]:
"""
 - 정답갯수, 오답갯수, 정답률, 오답률 출력
"""
### 예측 결과의 모든 행에서 -> 가장 높은 값을 가지는 열의 위치값을 추출하기
# - axis=1 : 열을 의미 (열 중에 가장 높은 값을 가지는 위치값 추출)
pred_classes = np.argmax(test_pred, axis=1)

### 정답갯수 확인하기
correct = pred_classes == test_target
correct_count = np.sum(correct)

### 오답갯수 확인하기
incorrect_count = len(test_pred) - correct_count

### 정답률 확인하기
correct_rate = correct_count / len(test_pred)

### 오답률 확인하기
incorrect_rate = incorrect_count / len(test_pred)

print(f"정답 갯수 : {correct_count}")
print(f"정답률 : {correct_rate*100}%")
print(f"오답 갯수 : {incorrect_count}")
print(f"오답률 : {incorrect_rate*100}&")

정답 갯수 : 8525
정답률 : 85.25%
오답 갯수 : 1475
오답률 : 14.75&


In [19]:
Ocnt = 0
Xcnt = 0
for i in range(len(test_target)):
    if np.argmax(test_pred[i]) == test_target[i]:
       Ocnt = Ocnt + 1
    
    else:
        Xcnt = Xcnt + 1
       
print(f"정답갯수 : {Ocnt}, 오답갯수 : {Xcnt}, 정답률 : {Ocnt/len(test_target)*100}%, 오답률 : {Xcnt/len(test_target)*100}%")

정답갯수 : 8525, 오답갯수 : 1475, 정답률 : 85.25%, 오답률 : 14.75%


### 성능향상 방법: <훈련(fit)중 최적화 시점의 모델 저장하기>

In [ ]:
"""
<최적화 시점의 모델 저장>
 - 훈련 반복(fit) 중 가장 좋은 시점(손실이 적거나, 더이상 손실이 낮아지지 않는 시점)의
   모델을 저장해서 사용하는 방법을 의미합니다.
   
<콜백(Callback) 함수란?>
 - 모델 훈련 중에 특정 작업(함수)을 호출(call)하여 수행하고자 할 때 사용
 - 훈련 시에 fit 함수에 적용하여 사용합니다.
 - 훈련 중에 발생시키는 함수라는 의미로 -> "이벤트(event) 함수"라고도 칭합니다.
 - 별도의 계층은 아니며, fit함수의 속성(매개변수)로 사용됩니다.


<최적화 시점의 모델 저장 및 훈련 종료를 위한 콜백(Callback) 함수 종류>
 1. ModelCheckpoint() 함수 : 모델(model) 확인(check) 시점(point) 확인 함수
  - epoch 마다 모델을 저장하는 기능을 수행하는 함수
  - 단, 이전 epoch에서 실행된 훈련 성능보다 높아진 경우에만 저장됨
 
 2. EarlyStopping() 함수 : 조기(earaly) 종료(stopping) 함수
  - 훈련이 더 이상 좋와지지 않으면(loss 기준) 훈련을 종료시키는 함수
  - "일반적으로 ModelCheckpoint()와 함께 사용함"
"""



In [20]:
model = Sequential([
    Input(shape=(784, )),
    Dense(units=100, activation="sigmoid", name="Input_layer"),
    Dropout(0.5),
    Dense(units=50, activation="relu", name="Hidden_layer"),
    Dropout(0.5),
    Dense(units=10, activation="softmax", name="Output_layer")
])

adam = keras.optimizers.Adam(
    learning_rate = 0.001,
    name="Output_layer"
)

model.compile(
    optimizer=adam,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

In [21]:
### 모델 저장시키는 콜백함수 정의하기
# - 라이브러리 임포트 해도 됨
modelcheckpoint = keras.callbacks.ModelCheckpoint(
    # 저장할 위치 지정
    "./model/model_all.h5",
    
    # epoch 마다 훈련 모델 자동 저장 여부 설정
    # - save_best_only = True : 모델 성능이 좋아질 때만 자동 저장
    # - svae_best_only = False : epoch 마다 저장
    save_best_only = True
)

modelcheckpoint


In [22]:
model.fit(
    train_scaled,
    train_target,
    validation_data=(val_scaled, 
                     val_target),
    epochs=10, 
    batch_size=32, 
    callbacks=[modelcheckpoint]
)

Epoch 1/10
1500/1500 [==============================] - 4s 2ms/step - loss: 0.9550 - accuracy: 0.6508 - val_loss: 0.5317 - val_accuracy: 0.8133
Epoch 2/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.6273 - accuracy: 0.7792 - val_loss: 0.4529 - val_accuracy: 0.8328
Epoch 3/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5702 - accuracy: 0.8029 - val_loss: 0.4363 - val_accuracy: 0.8370
Epoch 4/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5363 - accuracy: 0.8141 - val_loss: 0.4163 - val_accuracy: 0.8428
Epoch 5/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5101 - accuracy: 0.8242 - val_loss: 0.3994 - val_accuracy: 0.8514
Epoch 6/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4959 - accuracy: 0.8272 - val_loss: 0.3847 - val_accuracy: 0.8562
Epoch 7/10
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4832 - accuracy: 0.8326 - val_loss: 0.3762 - val_accuracy:

In [23]:
### 저장된 베스트 모델 불러들여서 예측
model_all = keras.models.load_model("./model/model_all.h5")
model_all.predict(test_scaled_2d)

313/313 [==============================] - 0s 1ms/step


array([[2.0749366e-07, 1.3891129e-09, 1.7133487e-11, ..., 1.1439994e-01,
        6.5770648e-05, 8.5049367e-01],
       [2.9224118e-05, 1.9466158e-09, 9.3810886e-01, ..., 3.6041575e-16,
        5.4346202e-07, 1.0785182e-14],
       [1.8249793e-08, 9.9999869e-01, 1.5553696e-09, ..., 1.1138210e-11,
        1.2448782e-13, 6.3424613e-13],
       ...,
       [4.5441731e-04, 5.3567645e-10, 4.2980014e-06, ..., 1.3798378e-07,
        9.9906534e-01, 5.1034743e-07],
       [4.5118861e-07, 9.9989820e-01, 1.2621970e-08, ..., 1.7657127e-09,
        8.0881777e-12, 4.1612502e-10],
       [3.4095590e-10, 4.4213212e-13, 3.1637076e-13, ..., 5.1825601e-03,
        3.5039620e-06, 8.7023400e-06]], dtype=float32)

### 훈련 자동 종료 시키는 -> 콜백함수 사용하기

In [ ]:
"""
 *** 최적의 모델 저장 및 훈련 자동 종료는 함께 사용됩니다. ***
"""

In [24]:
model = Sequential([
    Input(shape=(784, )),
    Dense(units=100, activation="sigmoid", name="Input_layer"),
    Dense(units=50, activation="relu", name="Hidden_layer"),
    Dense(units=10, activation="softmax", name="Output_layer")
])

adam = keras.optimizers.Adam(
    learning_rate = 0.001,
    name="Output_layer"
)

model.compile(
    optimizer=adam,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

In [25]:
### 모델 저장시키는 콜백함수 정의하기
# - 라이브러리 임포트 해도 됨
modelcheckpoint = keras.callbacks.ModelCheckpoint(
    # 저장할 위치 지정
    "./model/model_all.h5",
    
    # epoch 마다 훈련 모델 자동 저장 여부 설정
    # - save_best_only = True : 모델 성능이 좋아질 때만 자동 저장
    # - svae_best_only = False : epoch 마다 저장
    save_best_only = True
)

modelcheckpoint


In [26]:
### 훈련 자동 종료 시키는 콜백함수 정의
# - 라이브러리 임포트 해도 됨
earlystopping = keras.callbacks.EarlyStopping(
    # 더 이상 좋아지지 않는 epoch의 갯수 지정
    # - 가장 좋은 시점의 epoch이후 2번 더 수행 후
    #  - 그래도 좋아지지 않으면 종료시킨다
    patience = 2,
    
    ### 종료 시점에 가장 낮은 검증 손실일 때의 하이퍼 파라미터로 모델 업데이트 시킴
    # - 가중치 업데이트
    restore_best_weights=True
)

earlystopping

In [27]:
history = model.fit(
    train_scaled,
    train_target,
    validation_data=(val_scaled, 
                     val_target),
    epochs=1000, 
    batch_size=128, 
    callbacks=[modelcheckpoint, earlystopping]
)

Epoch 1/1000
375/375 [==============================] - 2s 4ms/step - loss: 0.7302 - accuracy: 0.7591 - val_loss: 0.4779 - val_accuracy: 0.8284
Epoch 2/1000
375/375 [==============================] - 1s 3ms/step - loss: 0.4394 - accuracy: 0.8438 - val_loss: 0.4134 - val_accuracy: 0.8507
Epoch 3/1000
375/375 [==============================] - 1s 3ms/step - loss: 0.3936 - accuracy: 0.8599 - val_loss: 0.3895 - val_accuracy: 0.8573
Epoch 4/1000
375/375 [==============================] - 1s 3ms/step - loss: 0.3721 - accuracy: 0.8659 - val_loss: 0.3906 - val_accuracy: 0.8578
Epoch 5/1000
375/375 [==============================] - 1s 3ms/step - loss: 0.3520 - accuracy: 0.8726 - val_loss: 0.3575 - val_accuracy: 0.8683
Epoch 6/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.3339 - accuracy: 0.8788 - val_loss: 0.3471 - val_accuracy: 0.8728
Epoch 7/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.3227 - accuracy: 0.8819 - val_loss: 0.3495 - val_accuracy:

In [28]:
"""
 - 최적의 모델 불러들이기
 - 훈련 성능 평가
 - 검증 성능 평가
"""
model_all = keras.models.load_model("./model/model_all.h5")
train_score = model_all.evaluate(train_scaled, train_target)
val_score = model_all.evaluate(val_scaled, val_target)

print(f"훈련 : {train_score[0]} / {train_score[1]}")
print(f"검증 : {val_score[0]} / {val_score[1]}")

375/375 [==============================] - 1s 1ms/step - loss: 0.3150 - accuracy: 0.8848
훈련 : 0.2514786124229431 / 0.9061874747276306
검증 : 0.31496286392211914 / 0.8847500085830688


### 성능향상 : 성능규제(Dropout) 방법

In [ ]:
"""
<성능 규제>
 - 성능(과적합 여부 포함)을 높이기 위한 방법
 - 보통 "전처리 계층"을 사용하게 됩니다.
 - 전처리 계층은 훈련에 영향을 미치지 않습니다.


<성능 규제 방법>
 * Dropout(드롭아웃) 방법을 사용함
  - 훈련 과정 중에 일부 "특성"들을 랜덤하게 제외 시켜서 과대/과소 적합을 해소하는 방법
  - 딥러닝에서 자주 사용되는 "전처리 계층"으로 성능 개선에 효율적으로 사용됨
  - 주로 과대적합이 발생한 경우에 사용하면 일반화에 효과가 있음
    (드롭아웃은 조금 멍청하게 만드는 개념입니다.)
 
 * Dropout 적용방법
  - 은닉계층(Hidden Layer)으로 추가하여 주로 사용됨
  - 제외시킬 값을 비율로 정의 합니다.
"""


In [29]:
model = Sequential([
    ### 입력계층 생성 및 추가하기
    Input(shape=(784, )),
    Dense(units=100, activation="sigmoid", name="Input_layer"),
    
    ### 주로 입력 계층 다음에 성능규제(Dropout) 계층을 사용합니다
    # - 전처리 계층
    # - Dropout(0.5) : 0.5는 50%를 의미함
    # - 사용되는 특성 중 랜덤하게 50%를 제거하고 다음 계층에 넘겨줌
    # *** 단, 제거는 하되 넘겨줄때 출력 갯수는 입력 갯수와 동일한 사이즈로 넘겨 줌 ***
    Dropout(0.5),
    
    ### 은닉계층 생성 및 추가하기
    Dense(units=50, activation="relu", name="Hidden_layer"),
    
    Dropout(0.5),
    
    ### 출력계층 생성 및 추가하기
    Dense(units=10, activation="softmax", name="Output_layer")
])

adam = keras.optimizers.Adam(
    learning_rate = 0.001,
    name="Output_layer"
)

model.compile(
    optimizer=adam,
    loss = "sparse_categorical_crossentropy",
    metrics = "accuracy"
)

In [30]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (Dense)         (None, 100)               78500     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 Hidden_layer (Dense)        (None, 50)                5050      
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 Output_layer (Dense)        (None, 10)                510       
                                                                 
Total params: 84,060
Trainable params: 84,060
Non-trainable params: 0
_________________________________________________________________


In [31]:
history = model.fit(
    train_scaled,
    train_target,
    validation_data=(val_scaled, 
                     val_target),
    epochs=1000, 
    batch_size=128, 
    callbacks=[modelcheckpoint, earlystopping]
)

Epoch 1/1000
375/375 [==============================] - 1s 3ms/step - loss: 1.2288 - accuracy: 0.5495 - val_loss: 0.6203 - val_accuracy: 0.7745
Epoch 2/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.7389 - accuracy: 0.7342 - val_loss: 0.5202 - val_accuracy: 0.8098
Epoch 3/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.6414 - accuracy: 0.7717 - val_loss: 0.4827 - val_accuracy: 0.8227
Epoch 4/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.5945 - accuracy: 0.7907 - val_loss: 0.4488 - val_accuracy: 0.8367
Epoch 5/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.5640 - accuracy: 0.8042 - val_loss: 0.4338 - val_accuracy: 0.8416
Epoch 6/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.5426 - accuracy: 0.8126 - val_loss: 0.4188 - val_accuracy: 0.8457
Epoch 7/1000
375/375 [==============================] - 1s 2ms/step - loss: 0.5256 - accuracy: 0.8184 - val_loss: 0.4089 - val_accuracy:

In [32]:
"""
 - 최적의 모델 불러들이기
 - 훈련 성능 평가
 - 검증 성능 평가
"""
model_all = keras.models.load_model("./model/model_all.h5")
train_score = model_all.evaluate(train_scaled, train_target)
val_score = model_all.evaluate(val_scaled, val_target)

print(f"훈련 : {train_score[0]} / {train_score[1]}")
print(f"검증 : {val_score[0]} / {val_score[1]}")

375/375 [==============================] - 1s 2ms/step - loss: 0.3150 - accuracy: 0.8848
훈련 : 0.2514786124229431 / 0.9061874747276306
검증 : 0.31496286392211914 / 0.8847500085830688


In [ ]:
# TensorFlow 및 Keras에서 필요한 모듈 임포트
from tensorflow import keras
from keras.models import Sequential                   # 순차 모델 클래스
from keras.layers import Dense, Dropout              # 완전 연결층(Dense), 드롭아웃(Dropout) 레이어
from keras.regularizers import l2                    # L2 정규화 함수
from keras.callbacks import EarlyStopping            # 조기 종료 콜백 함수

# 1. 모델 구조 정의
model = Sequential([
    Dense(128, activation='relu', input_shape=(784,), kernel_regularizer=l2(0.001)),  # 입력층 + 첫 번째 은닉층
    Dropout(0.5),                                               # 첫 번째 드롭아웃 (50%)
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)), # 두 번째 은닉층
    Dropout(0.5),                                               # 두 번째 드롭아웃 (50%)
    Dense(10, activation='softmax')                             # 출력층: 클래스 수가 10개, 소프트맥스 활성화
])

# 2. 옵티마이저 설정: Adam
adam = keras.optimizers.Adam(
    learning_rate=0.001           # 낮은 학습률: 안정적인 학습을 위해
)

# 3. 모델 컴파일
model.compile(
    optimizer=adam,                          # Adam 옵티마이저 사용
    loss='sparse_categorical_crossentropy',# 정수형 라벨을 위한 손실 함수
    metrics=['accuracy']                   # 정확도를 평가 지표로 사용
)

# 4. 모델 구조 출력
model.summary()                            # 모델 레이어 구조, 파라미터 수 등을 출력

# 5. EarlyStopping 콜백 설정
early_stop = EarlyStopping(
    monitor='val_loss',        # 검증 손실을 기준으로 개선 여부 판단
    patience=5,                # 5 epoch 동안 성능 향상이 없으면 학습 중단
    restore_best_weights=True # 가장 성능 좋았던 가중치를 복원
)


# 6. 모델 학습
history = model.fit(
    train_scaled, train_target,                     # 훈련 데이터
    validation_data=(val_scaled, val_target),      # 검증 데이터
    epochs=100,                          # 최대 100 epoch 학습
    batch_size=64,                       # 배치 사이즈 64로 학습
    callbacks=[early_stop]               # 조기 종료 콜백 적용
)


In [ ]:
train_score = model.evaluate(train_scaled, train_target)
val_score = model.evaluate(val_scaled, val_target)